Capstone Project

Description of the problem:

	Given the features of a sample of biopsied breast mass, can we 
    accurately tell if this mass is benign or malignant?
    
Description of the dataset:

	Attribute Information:
1) ID number 

2) Diagnosis (M = malignant, B = benign) – This will be what we attempt to determine

Ten real-valued features are computed for each cell nucleus: 

a) radius (mean of distances from center to points on the perimeter) 

b) texture (standard deviation of gray-scale values) 

c) perimeter 

d) area 

e) smoothness (local variation in radius lengths) 

f) compactness (perimeter^2 / area - 1.0) 

g) concavity (severity of concave portions of the contour) 

h) concave points (number of concave portions of the contour) 

i) symmetry 

j) fractal dimension ("coastline approximation" - 1)


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

let's go ahead and import the data we will be using and refer to it as df

In [6]:
df = pd.read_csv("data.csv")

Let's take a peek at the data 

In [7]:
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


looks like what we would expect so far - id's, m or b (though b isn't shown in the first 5) - this means either malignant or benign classification. Then the other columns that will be what we use to determine if it should be classified as m or b

In [8]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

above there are no null data items so the data looks pretty clean, let's go ahead and change the m's and b's a little bit. 

Let's see what our data looks like as far as b vs m

In [9]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

So the data has mostly benign items

above when we peeked at the data we saw unnamed 32 and this does not seem relevant to us. Let's go ahead and remove this. 

In [10]:
df.drop(['Unnamed: 32', 'id'], axis = 1, inplace = True)

In [11]:
df.head(5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Looks like that did the trick

In our assignments we had the dependent and independent variables. Here the diagnosis is what we are going to try to predict, so they will be dependent. 

In [12]:
x = df.drop('diagnosis', axis = 1)
y = df.diagnosis

Now the data is split up between our independent and dependent items

It would be ideal for us to go ahead and transform our dependent items to 0 and 1 instead of M and B

In [13]:
labeler = LabelEncoder()
y = labeler.fit_transform(y)

In [14]:
y[:5]

array([1, 1, 1, 1, 1])

Looks good! Our M's are now represented by '1'

Let's go ahead and split the dataset into a training and test set - 80 20 should work 

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Here we're making "original" sets so at the end we can take a look and verify for ourselves the classifier is working (The scaler will make that a little bit difficult)

In [16]:
x_train_orig = x_train
x_test_orig = x_test
y_train_orig = y_train
y_test_orig = y_test

Let's go ahead and scale the data into some smaller numbers as this may affect the weights over time

In [17]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [18]:
x_train

array([[-1.15036482, -0.39064196, -1.12855021, ..., -0.75798367,
        -0.01614761, -0.38503402],
       [-0.93798972,  0.68051405, -0.94820146, ..., -0.60687023,
         0.09669004, -0.38615797],
       [ 0.574121  , -1.03333557,  0.51394098, ..., -0.02371948,
        -0.20050207, -0.75144254],
       ...,
       [-1.32422924, -0.20048168, -1.31754581, ..., -0.97974953,
        -0.71542314, -0.11978123],
       [-1.24380987, -0.2245526 , -1.28007609, ..., -1.75401433,
        -1.58157125, -1.00601779],
       [-0.73694129,  1.14989702, -0.71226578, ..., -0.27460457,
        -1.25895095,  0.21515662]])

Looks like we are making progress and we are about ready to start looking at our neural network! 

In [19]:
df.shape

(569, 31)

so our original has 31 columns, with it broken out x has 30 columns and y has 1
this means our input layer should have 30

we're starting out pretty simple so let's just create a sequential model

below he_uniform is used to draw samples from a uniform distrubution

relu outputs the input directly if it is positive, otherwise it outputs 0. It helps to solve the vanishing gradient issue and helps to speed up the training - for a classification issue like this it should be useful

input_dim is the 30 inputs i.e. the 30 columns of information for each item

Glorot initialization helps to alleviate the proglem of vanishing gradients by setting the initial weights as distribution where the variance of the distribution is dependent on the number of input and output layers. It is designed for sigmoid functions 

sigmoid used for binary as it maps input values to a value between 0 and 1

In [20]:
model = Sequential()
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 279       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 1)                 10        
                                                                 
Total params: 379
Trainable params: 379
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [23]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
10/10 [==============================] - 1s 1ms/step - loss: 0.7120 - accuracy: 0.5099
Epoch 2/10
10/10 [==============================] - 0s 889us/step - loss: 0.6136 - accuracy: 0.6505
Epoch 3/10
10/10 [==============================] - 0s 889us/step - loss: 0.5362 - accuracy: 0.7626
Epoch 4/10
10/10 [==============================] - 0s 889us/step - loss: 0.4729 - accuracy: 0.8484
Epoch 5/10
10/10 [==============================] - 0s 889us/step - loss: 0.4176 - accuracy: 0.8901
Epoch 6/10
10/10 [==============================] - 0s 889us/step - loss: 0.3707 - accuracy: 0.8967
Epoch 7/10
10/10 [==============================] - 0s 778us/step - loss: 0.3302 - accuracy: 0.9099
Epoch 8/10
10/10 [==============================] - 0s 889us/step - loss: 0.2960 - accuracy: 0.9165
Epoch 9/10
10/10 [==============================] - 0s 889us/step - loss: 0.2678 - accuracy: 0.9275
Epoch 10/10
10/10 [==============================] - 0s 778us/step - loss: 0.2433 - accuracy: 0.9275


That only took seconds and was very quick! Looks like it is up to 92% accuracy on the test - it could always be overfitting and we don't know it yet, but since it was quick and we only did 10 epochs, let's try to do a few more.

In [24]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 0s 889us/step - loss: 0.2243 - accuracy: 0.9341
Epoch 2/50
10/10 [==============================] - 0s 889us/step - loss: 0.2078 - accuracy: 0.9407
Epoch 3/50
10/10 [==============================] - 0s 889us/step - loss: 0.1930 - accuracy: 0.9451
Epoch 4/50
10/10 [==============================] - 0s 889us/step - loss: 0.1799 - accuracy: 0.9451
Epoch 5/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1681 - accuracy: 0.9451
Epoch 6/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1575 - accuracy: 0.9495
Epoch 7/50
10/10 [==============================] - 0s 778us/step - loss: 0.1487 - accuracy: 0.9516
Epoch 8/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1408 - accuracy: 0.9560
Epoch 9/50
10/10 [==============================] - 0s 889us/step - loss: 0.1337 - accuracy: 0.9582
Epoch 10/50
10/10 [==============================] - 0s 778us/step - loss: 0.1275 - accuracy: 0.9626
Epoch

let's see how this works on our test data 98.90% seems pretty good!

In [25]:
y_pred = model.predict(x_test)

4/4 [==============================] - 0s 667us/step


Since we want to classify between two items, we can separate the predictions in true or false

In [26]:
y_pred = (y_pred > 0.5)

In [27]:
print(y_pred)

[[ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print (score)

[[64  3]
 [ 2 45]]
0.956140350877193


In [29]:
y_test.size

114

In [30]:
print(65+2+4+43)

114


In [31]:
np.count_nonzero(y_test)

47

Doing some double checking just to be sure - y_test has 114 items in it, and our confusion matrix has a sum of 114. We can be confident the confusion matrix is showing everything. Here we see there were:

the positives are 0's or B
65 true positives, 2 false positives

negatives are 1's or M
4 false negatives, 43 true negatives

The results here seem promising - there were only 2 false positives and 4 false negatives, only 6 items were identified incorrectly.

In [32]:
print(score)

0.956140350877193


In [33]:
y_test

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0])

In [34]:
x_test

array([[-0.20175604,  0.3290786 , -0.13086754, ...,  1.3893291 ,
         1.08203284,  1.54029664],
       [-0.25555773,  1.46763319, -0.31780437, ..., -0.83369364,
        -0.73131577, -0.87732522],
       [-0.02619262, -0.8407682 , -0.09175081, ..., -0.49483785,
        -1.22080864, -0.92115937],
       ...,
       [ 1.71811488,  0.09318356,  1.7286186 , ...,  1.57630515,
         0.20317063, -0.15406178],
       [ 1.18859296,  0.34352115,  1.19333694, ...,  0.56019755,
         0.26991966, -0.27320074],
       [ 0.26263752, -0.58080224,  0.28459338, ..., -0.19383705,
        -1.15564888,  0.11231497]])

In [35]:
x_test_orig

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
512,13.40,20.52,88.64,556.7,0.11060,0.14690,0.14450,0.08172,0.2116,0.07325,...,16.41,29.66,113.30,844.4,0.15740,0.38560,0.51060,0.20510,0.3585,0.11090
457,13.21,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,14.35,34.23,91.29,632.9,0.12890,0.10630,0.13900,0.06005,0.2444,0.06788
439,14.02,15.66,89.59,606.5,0.07966,0.05581,0.02087,0.02652,0.1589,0.05586,...,14.91,19.31,96.53,688.9,0.10340,0.10170,0.06260,0.08216,0.2136,0.06710
298,14.26,18.17,91.22,633.1,0.06576,0.05220,0.02475,0.01374,0.1635,0.05586,...,16.22,25.26,105.80,819.7,0.09445,0.21670,0.15650,0.07530,0.2636,0.07676
37,13.03,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.02923,0.1467,0.05863,...,13.30,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,17.42,25.56,114.50,948.0,0.10060,0.11460,0.16820,0.06597,0.1308,0.05866,...,18.07,28.07,120.40,1021.0,0.12430,0.17930,0.28030,0.10990,0.1603,0.06818
519,12.75,16.70,82.51,493.8,0.11250,0.11170,0.03880,0.02995,0.2120,0.06623,...,14.45,21.74,93.63,624.1,0.14750,0.19790,0.14230,0.08045,0.3071,0.08557
432,20.18,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.12590,0.1724,0.06053,...,22.03,25.07,146.00,1479.0,0.16650,0.29420,0.53080,0.21730,0.3032,0.08075
516,18.31,20.58,120.80,1052.0,0.10680,0.12480,0.15690,0.09451,0.1860,0.05941,...,21.86,26.20,142.20,1493.0,0.14920,0.25360,0.37590,0.15100,0.3074,0.07863


In [36]:
y_test_orig

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0])

So row 512 was marked as M or as 1

In [37]:
y_pred

array([[ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [

so to verify we are interpreting the results correctly, the 2nd item is false and should be B or 0

And upon checking the original data, this is true - I just wanted to do this check to make give myself a sanity check that the classifier is working as intended and I understand what is being interpreted.

let's see if adding more layers may increase our accuracy

In [38]:
model = Sequential()
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 9)                 279       
                                                                 
 dense_4 (Dense)             (None, 9)                 90        
                                                                 
 dense_5 (Dense)             (None, 9)                 90        
                                                                 
 dense_6 (Dense)             (None, 9)                 90        
                                                                 
 dense_7 (Dense)             (None, 1)                 10        
                                                                 
Total params: 559
Trainable params: 559
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [41]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 1s 889us/step - loss: 0.6957 - accuracy: 0.3670
Epoch 2/50
10/10 [==============================] - 0s 1ms/step - loss: 0.6348 - accuracy: 0.4835
Epoch 3/50
10/10 [==============================] - 0s 889us/step - loss: 0.5932 - accuracy: 0.6330
Epoch 4/50
10/10 [==============================] - 0s 778us/step - loss: 0.5615 - accuracy: 0.7341
Epoch 5/50
10/10 [==============================] - 0s 1ms/step - loss: 0.5350 - accuracy: 0.7824
Epoch 6/50
10/10 [==============================] - 0s 1ms/step - loss: 0.5096 - accuracy: 0.8418
Epoch 7/50
10/10 [==============================] - 0s 1ms/step - loss: 0.4876 - accuracy: 0.8681
Epoch 8/50
10/10 [==============================] - 0s 889us/step - loss: 0.4680 - accuracy: 0.8857
Epoch 9/50
10/10 [==============================] - 0s 1ms/step - loss: 0.4487 - accuracy: 0.8989
Epoch 10/50
10/10 [==============================] - 0s 889us/step - loss: 0.4292 - accuracy: 0.9143
Epoch 11/

In [42]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 667us/step


In [43]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)

[[64  3]
 [ 2 45]]


b

In [44]:
print(score)

0.956140350877193


This did essentially the same. It does not seem adding two more layers really did all that much. Maybe as it is a pretty small data set this is not helping much?

the training accuracy was much higher - maybe there is some overfitting going on? Perhaps if we add dropout

In [45]:
model = Sequential()
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dropout(0.5))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=9,kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 9)                 279       
                                                                 
 dropout (Dropout)           (None, 9)                 0         
                                                                 
 dense_9 (Dense)             (None, 9)                 90        
                                                                 
 dropout_1 (Dropout)         (None, 9)                 0         
                                                                 
 dense_10 (Dense)            (None, 9)                 90        
                                                                 
 dropout_2 (Dropout)         (None, 9)                 0         
                                                                 
 dense_11 (Dense)            (None, 9)                

In [47]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [48]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 1s 1ms/step - loss: 1.0035 - accuracy: 0.5099
Epoch 2/50
10/10 [==============================] - 0s 1ms/step - loss: 1.2076 - accuracy: 0.5275
Epoch 3/50
10/10 [==============================] - 0s 1ms/step - loss: 1.1254 - accuracy: 0.5473
Epoch 4/50
10/10 [==============================] - 0s 1ms/step - loss: 1.1407 - accuracy: 0.5626
Epoch 5/50
10/10 [==============================] - 0s 1ms/step - loss: 0.9159 - accuracy: 0.5538
Epoch 6/50
10/10 [==============================] - 0s 1ms/step - loss: 0.9438 - accuracy: 0.5890
Epoch 7/50
10/10 [==============================] - 0s 889us/step - loss: 1.0062 - accuracy: 0.5407
Epoch 8/50
10/10 [==============================] - 0s 1ms/step - loss: 0.8105 - accuracy: 0.5626
Epoch 9/50
10/10 [==============================] - 0s 1ms/step - loss: 0.7914 - accuracy: 0.5890
Epoch 10/50
10/10 [==============================] - 0s 889us/step - loss: 0.7748 - accuracy: 0.5824
Epoch 11/50
10/

In [49]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 1ms/step


In [50]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[67  0]
 [47  0]]
0.5877192982456141


Yikes, okay that was a failed idea... the accuracy got worse after a certain point in the epochs on the training data, so I don't think adding dropout is a great idea in this regard.

maybe we can play around with the neurons in each layer since they are all the same at the moment

In [51]:
model = Sequential()
model.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=5,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=3,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [52]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 20)                620       
                                                                 
 dense_14 (Dense)            (None, 10)                210       
                                                                 
 dense_15 (Dense)            (None, 5)                 55        
                                                                 
 dense_16 (Dense)            (None, 3)                 18        
                                                                 
 dense_17 (Dense)            (None, 1)                 4         
                                                                 
Total params: 907
Trainable params: 907
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [54]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 1s 889us/step - loss: 0.5722 - accuracy: 0.6769
Epoch 2/50
10/10 [==============================] - 0s 889us/step - loss: 0.4264 - accuracy: 0.8484
Epoch 3/50
10/10 [==============================] - 0s 1ms/step - loss: 0.3242 - accuracy: 0.9143
Epoch 4/50
10/10 [==============================] - 0s 1ms/step - loss: 0.2623 - accuracy: 0.9275
Epoch 5/50
10/10 [==============================] - 0s 1ms/step - loss: 0.2188 - accuracy: 0.9451
Epoch 6/50
10/10 [==============================] - 0s 889us/step - loss: 0.1899 - accuracy: 0.9560
Epoch 7/50
10/10 [==============================] - 0s 889us/step - loss: 0.1649 - accuracy: 0.9648
Epoch 8/50
10/10 [==============================] - 0s 889us/step - loss: 0.1482 - accuracy: 0.9692
Epoch 9/50
10/10 [==============================] - 0s 889us/step - loss: 0.1338 - accuracy: 0.9714
Epoch 10/50
10/10 [==============================] - 0s 778us/step - loss: 0.1218 - accuracy: 0.9802
Epoch

In [55]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 667us/step


In [56]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[65  2]
 [ 0 47]]
0.9824561403508771


That is much improved! Let's play around with some other things that can be done such as adding dropout

In [57]:
model = Sequential()
model.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dropout(0.5))
model.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=5,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=3,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [58]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [59]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 1s 1ms/step - loss: 0.6898 - accuracy: 0.4857
Epoch 2/50
10/10 [==============================] - 0s 1ms/step - loss: 0.6787 - accuracy: 0.5714
Epoch 3/50
10/10 [==============================] - 0s 2ms/step - loss: 0.6413 - accuracy: 0.6176
Epoch 4/50
10/10 [==============================] - 0s 2ms/step - loss: 0.6048 - accuracy: 0.6637
Epoch 5/50
10/10 [==============================] - 0s 2ms/step - loss: 0.5671 - accuracy: 0.6769
Epoch 6/50
10/10 [==============================] - 0s 2ms/step - loss: 0.5297 - accuracy: 0.6879
Epoch 7/50
10/10 [==============================] - 0s 2ms/step - loss: 0.4848 - accuracy: 0.7121
Epoch 8/50
10/10 [==============================] - 0s 2ms/step - loss: 0.4387 - accuracy: 0.7077
Epoch 9/50
10/10 [==============================] - 0s 2ms/step - loss: 0.4296 - accuracy: 0.7582
Epoch 10/50
10/10 [==============================] - 0s 1ms/step - loss: 0.4023 - accuracy: 0.7890
Epoch 11/50
10/10 [

In [60]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 667us/step


In [61]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[65  2]
 [ 1 46]]
0.9736842105263158


Dropout didn't affect things that much this time - since it is training fast and accuracy was still fluctuating at the end, let's see if we get it more epochs

In [62]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 100)

Epoch 1/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 0.9736
Epoch 2/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.9802
Epoch 3/100
10/10 [==============================] - 0s 889us/step - loss: 0.0908 - accuracy: 0.9714
Epoch 4/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0978 - accuracy: 0.9714
Epoch 5/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0858 - accuracy: 0.9758
Epoch 6/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 0.9714
Epoch 7/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0783 - accuracy: 0.9736
Epoch 8/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0937 - accuracy: 0.9692
Epoch 9/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0944 - accuracy: 0.9692
Epoch 10/100
10/10 [==============================] - 0s 1ms/step - loss: 0.0894 - accuracy: 0.9604
Epoch 1

In [63]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 1ms/step


In [64]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[65  2]
 [ 2 45]]
0.9649122807017544


It looks like the additional epochs and the dropout have combined a bit to give us a similar result, well 1.5% lower, but still it seems like this is pretty close. Perhaps one more dropout layer will improve things?

In [65]:
model = Sequential()
model.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dropout(0.5))
model.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=5,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=3,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [66]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [67]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 100)

Epoch 1/100
10/10 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5912
Epoch 2/100
10/10 [==============================] - 0s 1ms/step - loss: 0.6751 - accuracy: 0.5538
Epoch 3/100
10/10 [==============================] - 0s 889us/step - loss: 0.6579 - accuracy: 0.5912
Epoch 4/100
10/10 [==============================] - 0s 1ms/step - loss: 0.6566 - accuracy: 0.6176
Epoch 5/100
10/10 [==============================] - 0s 889us/step - loss: 0.6506 - accuracy: 0.6374
Epoch 6/100
10/10 [==============================] - 0s 889us/step - loss: 0.6331 - accuracy: 0.6374
Epoch 7/100
10/10 [==============================] - 0s 889us/step - loss: 0.6274 - accuracy: 0.6374
Epoch 8/100
10/10 [==============================] - 0s 1ms/step - loss: 0.6017 - accuracy: 0.6374
Epoch 9/100
10/10 [==============================] - 0s 889us/step - loss: 0.6058 - accuracy: 0.6374
Epoch 10/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5874 - accuracy: 0.6374

In [68]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 1ms/step


In [69]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[66  1]
 [ 2 45]]
0.9736842105263158


slightly better. I am curious if more epochs may be the answer when there is more dropout, as more changes for different things to "fire" or be learned from might help things. Let's try to double the epochs

In [70]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 200)

Epoch 1/200
10/10 [==============================] - 0s 889us/step - loss: 0.2166 - accuracy: 0.9670
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1945 - accuracy: 0.9692
Epoch 3/200
10/10 [==============================] - 0s 889us/step - loss: 0.2409 - accuracy: 0.9626
Epoch 4/200
10/10 [==============================] - 0s 889us/step - loss: 0.2161 - accuracy: 0.9648
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2159 - accuracy: 0.9670
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2158 - accuracy: 0.9604
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2060 - accuracy: 0.9802
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2230 - accuracy: 0.9670
Epoch 9/200
10/10 [==============================] - 0s 889us/step - loss: 0.2230 - accuracy: 0.9538
Epoch 10/200
10/10 [==============================] - 0s 889us/step - loss: 0.2044 - accuracy: 0.9714

In [71]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 1ms/step


In [72]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[66  1]
 [ 2 45]]
0.9736842105263158


Darn, it looks like the winner is still no dropout layer. Curious if the one dropout layer does any better with double the epochs

In [73]:
model = Sequential()
model.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dropout(0.5))
model.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=5,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=3,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [74]:
model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

In [75]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 200)

Epoch 1/200
10/10 [==============================] - 1s 889us/step - loss: 0.7222 - accuracy: 0.3560
Epoch 2/200
10/10 [==============================] - 0s 889us/step - loss: 0.7018 - accuracy: 0.4593
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 0.6848 - accuracy: 0.5473
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 0.6766 - accuracy: 0.6418
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 0.6646 - accuracy: 0.7033
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 0.6504 - accuracy: 0.7275
Epoch 7/200
10/10 [==============================] - 0s 889us/step - loss: 0.6384 - accuracy: 0.7780
Epoch 8/200
10/10 [==============================] - 0s 889us/step - loss: 0.6179 - accuracy: 0.8198
Epoch 9/200
10/10 [==============================] - 0s 889us/step - loss: 0.6084 - accuracy: 0.8286
Epoch 10/200
10/10 [==============================] - 0s 889us/step - loss: 0.5757 - accuracy: 0.85

In [76]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 667us/step


In [77]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[64  3]
 [ 3 44]]
0.9473684210526315


In [78]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 500)

Epoch 1/500
10/10 [==============================] - 0s 1ms/step - loss: 0.1290 - accuracy: 0.9912
Epoch 2/500
10/10 [==============================] - 0s 1ms/step - loss: 0.1312 - accuracy: 0.9912
Epoch 3/500
10/10 [==============================] - 0s 1ms/step - loss: 0.1318 - accuracy: 0.9912
Epoch 4/500
10/10 [==============================] - 0s 1ms/step - loss: 0.1285 - accuracy: 0.9912
Epoch 5/500
10/10 [==============================] - 0s 1ms/step - loss: 0.1238 - accuracy: 0.9934
Epoch 6/500
10/10 [==============================] - 0s 889us/step - loss: 0.1273 - accuracy: 0.9912
Epoch 7/500
10/10 [==============================] - 0s 889us/step - loss: 0.1259 - accuracy: 0.9934
Epoch 8/500
10/10 [==============================] - 0s 889us/step - loss: 0.1291 - accuracy: 0.9890
Epoch 9/500
10/10 [==============================] - 0s 889us/step - loss: 0.1248 - accuracy: 0.9912
Epoch 10/500
10/10 [==============================] - 0s 889us/step - loss: 0.1273 - accuracy: 0.9890

In [79]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 667us/step


In [80]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[63  4]
 [ 2 45]]
0.9473684210526315


I went a bit overboard on the last one and did 500 - it seems this one is still the best model, and I'm unsure if the 95 to 97 range is more of some benefit in only having 100 epochs vs 500 and 100 is the sweet spot, or if there is some element of chance going on here. Regardless, the model with:

model = Sequential()
model.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',input_dim=30))
model.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=5,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=3,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])
              
Appears to be the best from what we have selected, with only 2 of the items missclassified. Not bad! 

Let's see if we can achieve any different results with RNN / if RNN may work

In [ ]:
import tensorflow as tf
from tensorflow import keras

Really simple RNN model, don't need to specify length of inputs as a rnn can process any number of steps

In [83]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="mse", optimizer=optimizer)

In [84]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
10/10 [==============================] - 2s 5ms/step - loss: 0.5541
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3475
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2414
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.1541
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.1118
Epoch 6/10
10/10 [==============================] - 0s 6ms/step - loss: 0.0905
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0765
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0669
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0591
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0531


In [86]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [87]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[62  5]
 [ 2 45]]
0.9385964912280702


Above we did mse just to see how that would look, but I want to try with binary crossentropy loss

In [96]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [97]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
10/10 [==============================] - 2s 5ms/step - loss: 2.1620
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.4757
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3903
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3710
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3266
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3105
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2754
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2318
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2261
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2212


In [98]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [99]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[64  3]
 [ 2 45]]
0.956140350877193


Not bad, but those losses are still pretty large and they were shrinking fast, let's try on more epochs

In [100]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [101]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 50)

Epoch 1/50
10/10 [==============================] - 2s 5ms/step - loss: 2.9798
Epoch 2/50
10/10 [==============================] - 0s 5ms/step - loss: 2.3053
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 1.8142
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 1.5675
Epoch 5/50
10/10 [==============================] - 0s 5ms/step - loss: 1.2799
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 1.1021
Epoch 7/50
10/10 [==============================] - 0s 6ms/step - loss: 0.8948
Epoch 8/50
10/10 [==============================] - 0s 5ms/step - loss: 0.6885
Epoch 9/50
10/10 [==============================] - 0s 6ms/step - loss: 0.5940
Epoch 10/50
10/10 [==============================] - 0s 5ms/step - loss: 0.5550
Epoch 11/50
10/10 [==============================] - 0s 6ms/step - loss: 0.5302
Epoch 12/50
10/10 [==============================] - 0s 5ms/step - loss: 0.5127
Epoch 13/50
10/10 [==============================

In [102]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [103]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[63  4]
 [ 4 43]]
0.9298245614035088


I tried this a couple times with the 50 epochs and got a range from 92-95%, maybe with more epochs it will get to a state that is more consistent. If that is the case maybe we can look at adding dropout to make it more consistent over less epochs?

In [104]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [105]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 200)

Epoch 1/200
10/10 [==============================] - 2s 5ms/step - loss: 2.0816
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 0.7418
Epoch 3/200
10/10 [==============================] - 0s 6ms/step - loss: 0.4672
Epoch 4/200
10/10 [==============================] - 0s 6ms/step - loss: 0.3984
Epoch 5/200
10/10 [==============================] - 0s 5ms/step - loss: 0.3243
Epoch 6/200
10/10 [==============================] - 0s 6ms/step - loss: 0.3208
Epoch 7/200
10/10 [==============================] - 0s 5ms/step - loss: 0.3140
Epoch 8/200
10/10 [==============================] - 0s 6ms/step - loss: 0.3062
Epoch 9/200
10/10 [==============================] - 0s 5ms/step - loss: 0.2725
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 0.2651
Epoch 11/200
10/10 [==============================] - 0s 5ms/step - loss: 0.2603
Epoch 12/200
10/10 [==============================] - 0s 5ms/step - loss: 0.2557
Epoch 13/200
10/10 [=================

In [106]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [107]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[66  1]
 [ 1 46]]
0.9824561403508771


That does seem to be pretty good! The loss got to a relatively consistent spot and eventually it got to 98% accuracy. Let's try with dropout of .2 and see if this impacts things

In [108]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.Dropout(rate=0.2),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [109]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 100)

Epoch 1/100
10/10 [==============================] - 3s 7ms/step - loss: 1.1727
Epoch 2/100
10/10 [==============================] - 0s 6ms/step - loss: 0.9664
Epoch 3/100
10/10 [==============================] - 0s 6ms/step - loss: 0.7305
Epoch 4/100
10/10 [==============================] - 0s 6ms/step - loss: 0.6392
Epoch 5/100
10/10 [==============================] - 0s 6ms/step - loss: 0.5155
Epoch 6/100
10/10 [==============================] - 0s 6ms/step - loss: 0.5940
Epoch 7/100
10/10 [==============================] - 0s 6ms/step - loss: 0.4726
Epoch 8/100
10/10 [==============================] - 0s 6ms/step - loss: 0.4713
Epoch 9/100
10/10 [==============================] - 0s 6ms/step - loss: 0.3802
Epoch 10/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4613
Epoch 11/100
10/10 [==============================] - 0s 6ms/step - loss: 0.3699
Epoch 12/100
10/10 [==============================] - 0s 6ms/step - loss: 0.4695
Epoch 13/100
10/10 [=================

In [110]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [111]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[65  2]
 [ 1 46]]
0.9736842105263158


hm looks like it went down a tad, so dropout might not be doing much. Maybe over more epochs as above?

In [112]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.Dropout(rate=0.5),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.Dropout(rate=0.5),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [113]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 200)

Epoch 1/200
10/10 [==============================] - 2s 7ms/step - loss: 1.1470
Epoch 2/200
10/10 [==============================] - 0s 7ms/step - loss: 1.2253
Epoch 3/200
10/10 [==============================] - 0s 6ms/step - loss: 0.8908
Epoch 4/200
10/10 [==============================] - 0s 6ms/step - loss: 0.7089
Epoch 5/200
10/10 [==============================] - 0s 6ms/step - loss: 0.7397
Epoch 6/200
10/10 [==============================] - 0s 6ms/step - loss: 0.6361
Epoch 7/200
10/10 [==============================] - 0s 7ms/step - loss: 0.5241
Epoch 8/200
10/10 [==============================] - 0s 7ms/step - loss: 0.6307
Epoch 9/200
10/10 [==============================] - 0s 6ms/step - loss: 0.5397
Epoch 10/200
10/10 [==============================] - 0s 6ms/step - loss: 0.4782
Epoch 11/200
10/10 [==============================] - 0s 7ms/step - loss: 0.4620
Epoch 12/200
10/10 [==============================] - 0s 6ms/step - loss: 0.4146
Epoch 13/200
10/10 [=================

In [114]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 2ms/step


In [115]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[64  3]
 [ 2 45]]
0.956140350877193


This time it actually went down, so perhaps dropout has not been useful.

Last thing is lets try to add one more layer.

In [116]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(10, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [117]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 200)

Epoch 1/200
10/10 [==============================] - 3s 7ms/step - loss: 1.2333
Epoch 2/200
10/10 [==============================] - 0s 7ms/step - loss: 0.7619
Epoch 3/200
10/10 [==============================] - 0s 7ms/step - loss: 0.5722
Epoch 4/200
10/10 [==============================] - 0s 7ms/step - loss: 0.4934
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 0.4370
Epoch 6/200
10/10 [==============================] - 0s 7ms/step - loss: 0.3852
Epoch 7/200
10/10 [==============================] - 0s 7ms/step - loss: 0.3662
Epoch 8/200
10/10 [==============================] - 0s 7ms/step - loss: 0.3318
Epoch 9/200
10/10 [==============================] - 0s 7ms/step - loss: 0.2948
Epoch 10/200
10/10 [==============================] - 0s 7ms/step - loss: 0.2536
Epoch 11/200
10/10 [==============================] - 0s 7ms/step - loss: 0.2443
Epoch 12/200
10/10 [==============================] - 0s 7ms/step - loss: 0.2385
Epoch 13/200
10/10 [=================

In [118]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 3ms/step


In [119]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[66  1]
 [ 1 46]]
0.9824561403508771


the loss was going up after about 150..

In [123]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(10, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [124]:
history = model.fit(x_train, y_train, batch_size = 50, epochs = 150)

Epoch 1/150
10/10 [==============================] - 3s 7ms/step - loss: 2.2346
Epoch 2/150
10/10 [==============================] - 0s 7ms/step - loss: 0.8194
Epoch 3/150
10/10 [==============================] - 0s 7ms/step - loss: 0.3176
Epoch 4/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2752
Epoch 5/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2648
Epoch 6/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2567
Epoch 7/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2203
Epoch 8/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2122
Epoch 9/150
10/10 [==============================] - 0s 7ms/step - loss: 0.2055
Epoch 10/150
10/10 [==============================] - 0s 7ms/step - loss: 0.1976
Epoch 11/150
10/10 [==============================] - 0s 7ms/step - loss: 0.1951
Epoch 12/150
10/10 [==============================] - 0s 7ms/step - loss: 0.1914
Epoch 13/150
10/10 [=================

In [125]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

4/4 [==============================] - 0s 3ms/step


In [126]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_test, y_pred)
print (cm)
print(score)

[[64  3]
 [ 1 46]]
0.9649122807017544


It does seem RNN is very comparable, and even gets up to 98% a bit more consistently. This is with a very simple model too. 